# Demo's for AZ-2005 (work in progress)

This demo will showcase RAG patterns using Azure Cognitive Search

In [ ]:
#r "nuget:Microsoft.Extensions.Logging, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Debug, 8.0.0"

#r "nuget:Microsoft.SemanticKernel, 1.16.2"
#r "nuget:Microsoft.SemanticKernel.Plugins.Core, 1.16.2-alpha"
#r "nuget:Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.16.2-alpha"
#r "nuget:Microsoft.SemanticKernel.Plugins.Memory, 1.16.2-alpha"
#r "nuget:dotenv.net, 3.0.0"

#r "nuget:Microsoft.KernelMemory.Core, 0.69.240727.1"

In [ ]:
using Microsoft.KernelMemory;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Memory;
using Microsoft.Extensions.Configuration;
using dotenv.net;
using dotenv.net.Utilities;

DotEnv.Load();

var openai_endpoint = EnvReader.GetStringValue("OPENAI_ENDPOINT");
var openai_key = EnvReader.GetStringValue("OPENAI_KEY");
var azureOpenAITextConfig = new AzureOpenAIConfig();
var azureOpenAIEmbeddingConfig = new AzureOpenAIConfig();
var azureAISearchConfigWithHybridSearch = new AzureAISearchConfig();

azureAISearchConfigWithHybridSearch.UseHybridSearch = true;

new ConfigurationBuilder()
            .AddJsonFile($"{System.IO.Directory.GetCurrentDirectory()}\\appsettings.dev.json")
            .Build()
            .BindSection("KernelMemory:Services:AzureOpenAIText", azureOpenAITextConfig)
            .BindSection("KernelMemory:Services:AzureOpenAIEmbedding", azureOpenAIEmbeddingConfig)
            .BindSection("KernelMemory:Services:AzureAISearch", azureAISearchConfigWithHybridSearch);

var memory = new KernelMemoryBuilder()
            .WithAzureOpenAITextGeneration(azureOpenAITextConfig)
            .WithAzureOpenAITextEmbeddingGeneration(azureOpenAIEmbeddingConfig)
            .WithAzureAISearchMemoryDb(azureAISearchConfigWithHybridSearch)
            .WithSearchClientConfig(new SearchClientConfig { MaxMatchesCount = 2, Temperature = 0, TopP = 0 })
            .Build<MemoryServerless>();


In [ ]:
// delete index
await memory.DeleteIndexAsync("demo_04");

// Import a file
await memory.ImportDocumentAsync(@"C:\_git\weslbo\AZ-2005\semantic-kernel.pdf", index: "demo_04");

In [ ]:
var answer = await memory.AskAsync("What is Semantic Kernel?", index:"demo_04");

answer.Result.ToString().DisplayAs("text/markdown");

foreach (var x in answer.RelevantSources)
{
    Console.WriteLine($"  * {x.SourceName} -- {x.Partitions.First().LastUpdate:D}");
}

var prompt = @"
            Question to Kernel Memory: {{$input}}

            Kernel Memory Answer: {{memory.ask}}

            If the answer is empty say 'I don't know', otherwise reply with a business mail to share the answer.
            ";answer